## Deep Neural Network from Scratch

**Following up from our last notebook, we will discuss applying Deep Neural Netowks for Binary and Multiclass classification for different datasets. Also, here we will lay the foundations for our modules associated with our different steps of the pipeline(like cost functions, activations, etc). Here, we will also go through all the advanced concepts of Neural networks, some of which includes:**

- Optimization techniques (Adam, RMSprop, Momentum)
- Batch Normalization
- Gradient Clipping
- Dropout, L1/L2 regularization
- Learning rate decay
- Early stopping
- Analysing results with different metrics (R2-score, f1-score, Confusion matrix, AUC-ROC curves)

_We will make different modules (.py files) from which we will import our functions for now, with classes for different tasks. Later we will set up classes for all our tasks._

### Classification task

In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

#### For this classification task, we will use plant growth data with binary labels growth_milestone

In [2]:
df = pd.read_csv("datasets/plant_growth_data.csv")
df

,Soil_Type,Sunlight_Hours,Water_Frequency,Fertilizer_Type,Temperature,Humidity,Growth_Milestone
0,loam,5.192294,bi-weekly,chemical,31.719602,61.591861,0
1,sandy,4.033133,weekly,organic,28.919484,52.422276,1
2,loam,8.892769,bi-weekly,none,23.179059,44.660539,0
3,loam,8.241144,bi-weekly,none,18.465886,46.433227,0
4,sandy,8.374043,bi-weekly,organic,18.128741,63.625923,0
...,...,...,...,...,...,...,...
188,sandy,5.652000,daily,none,28.000000,70.200000,0
189,clay,7.528000,weekly,chemical,30.500000,60.100000,1
190,loam,4.934000,bi-weekly,none,24.500000,61.700000,0
191,sandy,8.273000,daily,organic,27.900000,69.500000,1


In [3]:
# Encoding categorical columns

ohe = OneHotEncoder()
le = LabelEncoder()

df['Soil_Type'] = ohe.fit_transform(df[['Soil_Type']]).toarray()
df['Fertilizer_Type'] = ohe.fit_transform(df[['Fertilizer_Type']]).toarray()
df['Water_Frequency'] = le.fit_transform(df['Water_Frequency'])

df

,Soil_Type,Sunlight_Hours,Water_Frequency,Fertilizer_Type,Temperature,Humidity,Growth_Milestone
0,0.0,5.192294,0,1.0,31.719602,61.591861,0
1,0.0,4.033133,2,0.0,28.919484,52.422276,1
2,0.0,8.892769,0,0.0,23.179059,44.660539,0
3,0.0,8.241144,0,0.0,18.465886,46.433227,0
4,0.0,8.374043,0,0.0,18.128741,63.625923,0
...,...,...,...,...,...,...,...
188,0.0,5.652000,1,0.0,28.000000,70.200000,0
189,1.0,7.528000,2,1.0,30.500000,60.100000,1
190,0.0,4.934000,0,0.0,24.500000,61.700000,0
191,0.0,8.273000,1,0.0,27.900000,69.500000,1


In [4]:
# Dividing data to train and test datasets

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3, random_state=42)
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((135, 6), (135, 1)), ((58, 6), (58, 1)))

**Now, for this Deep Neural Network, we will define our custom Neural Network, where we just need to provide different NN architecture, and optimizer, loss, etc, and NN will train our NN based on the architecture and hyperparameters provided.**

**Weights initialization** function is defined <a href="weights_init.py" style="text-decoration: underline">here</a>, and **Forward propagation** function is defined <a href="forward_propagation.py" style="text-decoration: underline">here</a>.

In [5]:
from weights_init import specialInitialization
from forward_propagation import forward_prop
from back_propagation import back_propagation
from parameters_update import parameters_update

In [6]:
layers = 5 # We can change this parameter based on our NN architecture
layer_neurons = [6, 10, 16, 12, 4, 1]
# first number is the number of features in our input data, last number is the number of neurons in our output layer
# and, middle numbers are the number of hidden layer neurons in each layer.

layer_activations = ["relu", "tanh", "tanh", "sigmoid"] # activation functions for hidden layers, and we will decide activation function for the 
# output layer based on the loss function provided.

## This neural network has 5 layer(4 hidden, 1 outer layer) with 10, 16, 12, and 4 neurons respectively at each hidden layer.
## Input data has 6 features so this NN has 6 input neurons, and 1 output neuron as this is the case of binary classification(0/1)

parameters = specialInitialization(neuron_layers=layer_neurons)
learning_rate = 100

"""
forward_prop(X, y, layers, layer_neurons, parameters, activation, cost_function)

Based on the parameters provided, we will do the forward propagation step.

X: Training data
y: labels of training data
layers: Number of layers in NN
layer_neurons: List consisting of number of neurons at each layer(includes input, hidden and output layers).
parameters: weights and biases
layer_activations: activation function list for non-linearity at each hidden layer
cost_function: cost function for computing cost after each forward propagation.

Returns: Activations(a, z) and Cost value of this forward prop
"""
activations, cost = forward_prop(X_train, y_train, layers, layer_neurons, parameters, layer_activations, "bce")

print(f"Cost: {cost}")
activations.keys()

Cost: 12742.015724359475


dict_keys(['a_0', 'z_1', 'a_1', 'z_2', 'a_2', 'z_3', 'a_3', 'z_4', 'a_4', 'z_5', 'a_5'])

Till now, we have successfully initialized our weights, did forward propagation with our defined activation function, and also computed the cost with our type of cost function defined in the parameter of the function.

**Now, lets do the backpropagation and compute the gradients.** _To dive deep into Backpropagation, look into <a href="neural_network.ipynb" style="text-decoration: underline">this</a> file._

And <a href="back_propagation.py" style="text-decoration: underline">this</a> is how back propagation function is defined.

In [7]:
"""
back_propagation(y, layers, layer_neurons, parameters, activations, layer_activations, cost_function)

y, layers, layer_neurons, parameters, layer_activations, cost_function: same as defined in forward propagation.
activations: activations output from forward propagation

Returns: Gradient(da, dw, dw, db)
"""
gradients = back_propagation(y_train, layers, layer_neurons, parameters, activations, layer_activations, "bce")

for k in gradients.keys():
    print(k, gradients[k].shape)

da_5 (1, 135)
dz_5 (1, 135)
dw_5 (1, 4)
db_5 (1, 1)
da_4 (4, 135)
dz_4 (4, 135)
dw_4 (4, 12)
db_4 (4, 1)
da_3 (12, 135)
dz_3 (12, 135)
dw_3 (12, 16)
db_3 (12, 1)
da_2 (16, 135)
dz_2 (16, 135)
dw_2 (16, 10)
db_2 (16, 1)
da_1 (10, 135)
dz_1 (10, 135)
dw_1 (10, 6)
db_1 (10, 1)


**Now, after computing the gradients, the next step is to update the weights using the computed gradients.**

_<a href="parameters_update.py" style="text-decoration: underline">Here</a> is how **Gradient updation** function is defined._

In [8]:
"""
parameters_update(layers, parameters, gradients, learning_rate)

layers: total NN layers count (includes hidden and outer layer)
parameters: w, b values which is to be updated in this function
gradients: gradients values from backpropagation step
learning_rate: learning rate for updating parameters
"""
parameters = parameters_update(layers, parameters, gradients, learning_rate)

for p in parameters.keys():
    print(p, parameters[p].shape)

w1 (10, 6)
b1 (10, 1)
w2 (16, 10)
b2 (16, 1)
w3 (12, 16)
b3 (12, 1)
w4 (4, 12)
b4 (4, 1)
w5 (1, 4)
b5 (1, 1)


#### Henceforth, we have completed our first iteration of the Neural Network training.

**Now, we have to repeat this process for many iterations, and come with with a good value of parameters at the end.**

_For this, we will use any of the optimization algorithm._

In [14]:
from optimizer import batch_GD

parameters, costs = batch_GD(X_train, y_train, layers, layer_neurons, layer_activations, "bce", 0.001, 5000)

parameters

------------100-------------
------------200-------------
------------300-------------
------------400-------------
------------500-------------
------------600-------------
------------700-------------
------------800-------------
------------900-------------
------------1000-------------
------------1100-------------
------------1200-------------
------------1300-------------
------------1400-------------
------------1500-------------
------------1600-------------
------------1700-------------
------------1800-------------
------------1900-------------
------------2000-------------
------------2100-------------
------------2200-------------
------------2300-------------
------------2400-------------
------------2500-------------
------------2600-------------
------------2700-------------
------------2800-------------
------------2900-------------
------------3000-------------
------------3100-------------
------------3200-------------
------------3300-------------
------------3400---

{'w1': array([[ 0.16452054, -0.05095608, -0.04520188,  0.14706156,  0.09539093,
         -0.09135554],
        [-0.23339207,  0.24029214, -0.25022339, -0.33660281, -0.26019892,
          0.63169898],
        [ 0.55022776, -0.38032747,  0.15099073,  0.11066234, -0.32825384,
          0.08046303],
        [ 0.26729261,  0.39665863,  0.11934574,  0.20427017,  0.05470354,
         -0.25847901],
        [ 0.46509166,  0.4784919 ,  0.25261608,  0.16273505, -2.10537263,
          0.25370612],
        [ 0.06463764,  0.0112206 ,  0.31420166, -0.29636771,  0.09637643,
         -0.23010185],
        [ 0.20448436, -0.10700548, -0.05554317,  0.19051838, -0.10694203,
         -0.01590074],
        [-0.41266375, -1.05094424,  0.71234284,  0.00482166,  3.63576547,
         -0.49885969],
        [-0.23173431, -0.13579059,  0.36283603,  0.01102796, -0.0676508 ,
         -0.09252559],
        [ 0.01570173, -0.17564769, -0.09668451, -0.40337724,  0.71087541,
         -0.55047708]]),
 'b1': array([[-0.0045